<a href="https://colab.research.google.com/github/jbloewencolon/AI-News-Themes-of-the-Week/blob/main/AI_News_draft_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')  # One-time download for sentence splitting
nltk.download('stopwords')  # Download common 'stop words'

def get_ai_news_headlines():
    """ I fetch the latest AI news headlines using the News API. """

    api_key = "b506809c382c4befb258e4febc1fdf78"
    base_url = "https://newsapi.org/v2/everything"

    # Build the request parameters
    parameters = {
        'q': 'artificial intelligence',  # Your main search term
        'apiKey': api_key
    }

    response = requests.get(base_url, params=parameters)
    response.raise_for_status()  # Check for successful API response

    data = response.json()

    headlines = [article['title'] for article in data['articles']]
    return headlines

if __name__ == "__main__":
    latest_headlines = get_ai_news_headlines()
    print("This Week's Top AI News Headlines:")
    for headline in latest_headlines:
        print(headline)

def analyze_headlines(headlines):
    """ I process the headlines to find meaningful keywords"""
    all_words = []

    for headline in headlines:
        words = nltk.word_tokenize(headline)
        # Lowercase for simplicity
        words = [word.lower() for word in words if word.isalpha()]
        # Filter out stop words (the, is, a, etc.)
        stop_words = set(nltk.corpus.stopwords.words('english'))
        words = [w for w in words if not w in stop_words]

        all_words.extend(words)

    # Find the most frequent words
    freq_dist = nltk.FreqDist(all_words)
    top_keywords = [word for word, count in freq_dist.most_common(10)]  # Adjust the number for more/fewer keywords

    return top_keywords

if __name__ == "__main__":
    latest_headlines = get_ai_news_headlines()
    print("This Week's Top AI News Headlines:")
    for headline in latest_headlines:
        print(headline)
    keywords = analyze_headlines(latest_headlines)
    print("This Week's Top AI Keywords:")
    print(keywords)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


This Week's Top AI News Headlines:
Here's How Generative AI Depicts Queer People
Join The Verge at the 2024 Chicago Humanities Spring Festival
To Build a Better AI Supercomputer, Let There Be Light
Jon Stewart Confirms Apple Wouldn't Let Him Do Show on AI With FTC Chair
Artificial intelligence vs machine learning: what’s the difference?
[Removed]
[Removed]
On-Device AI Is a Whole New Way of Experiencing Artificial Intelligence - CNET
Microsoft Copilot Is Offering GPT-4 Turbo for Free: What to Know - CNET
Microsoft and OpenAI plan to build a $100 billion supercomputer to power artificial intelligence: report
[Removed]
Google wants to leverage AI to make weather forecasting more efficient
Utah Passes Artificial Intelligence Legislation
[Removed]
[Removed]
Europe is trying to regulate AI. That could backfire.
EU finally adopts AI Act, marking a ‘new era’ for artificial intelligence
Read AI raises $21M to bring connected intelligence to meetings, email, and messaging
Trump Says Humiliating